In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import librosa
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH_MODEL = '../model/model.h5'
SAMPLES_TO_CONSIDER = 22050 

In [3]:
def extraire_info_audio(file_path,num_mfcc=13, n_fft=2048, hop_length=512):
    # Découper L'audio pour assurer la cohérence de la longueur entre les différents fichiers
    signal, sample_rate = librosa.load(file_path)

    # Le nombre d'echantillon doit être supérieur à un certain seuil 
    if len(signal) >= SAMPLES_TO_CONSIDER:
        # Assurer la cohérence de la longueur du signal
        signal = signal[:SAMPLES_TO_CONSIDER]

        # Extraire les MFCCs
        MFCCs = librosa.feature.mfcc(signal, n_mfcc = num_mfcc, n_fft = n_fft, hop_length = hop_length)
        return MFCCs
    return []

In [4]:
class_names = [
        "right",
        "off",
        "left",
        "down",
        "_background_noise_",
        "up",
        "on"
    ]

def prediction(file_path):
    
    # Charger le modele entrainé
    model = load_model(PATH_MODEL)
    # charger l'audio pour le découper
    signal, sample_rate = librosa.load(file_path)
    MFCCs = extraire_info_audio(file_path)
    if len(MFCCs) > 0 :
        audio_array = tf.expand_dims(MFCCs.T.tolist(),0)
        audio_array = audio_array[..., np.newaxis]
        predictions = model.predict(audio_array)
        score = predictions[0]
        print(score)
        print(np.argmax(score))
        print("Valeur prédit est : " + class_names[np.argmax(score)] + " avec une confiance de "
              + str(100 * np.max(score)) +"%")
        print()


In [5]:
prediction('../audio_philippe.wav')
prediction('../off_1605688283167.wav')
prediction('../down.wav')
prediction('../left_1605688901391.wav')
prediction('../on.wav')
prediction('../up_1605687833098.wav')
prediction('../right_1605688777747.wav')

[0.01938195 0.05775663 0.04028965 0.02969119 0.0115528  0.7813863
 0.05994143]
5
Valeur prédit est : up avec une confiance de 78.13863158226013%

[0.01664314 0.5804005  0.05346676 0.00749971 0.01124552 0.31215245
 0.01859186]
1
Valeur prédit est : off avec une confiance de 58.04005265235901%

[3.4398269e-03 6.9495025e-03 7.6176681e-02 8.6130238e-01 1.5330128e-04
 2.9406283e-02 2.2572024e-02]
3
Valeur prédit est : down avec une confiance de 86.1302375793457%

[0.06625287 0.07249124 0.46868366 0.08532019 0.01992061 0.26013747
 0.02719401]
2
Valeur prédit est : left avec une confiance de 46.8683660030365%

[0.03046664 0.03126249 0.05361205 0.11231021 0.00546349 0.07302104
 0.6938641 ]
6
Valeur prédit est : on avec une confiance de 69.3864107131958%

[0.0515501  0.04921091 0.05635419 0.08053076 0.00951201 0.5897054
 0.16313665]
5
Valeur prédit est : up avec une confiance de 58.97054076194763%

[9.9990499e-01 6.4529068e-08 5.5478955e-05 3.8211851e-07 9.9146894e-07
 1.7223733e-06 3.6455294e-